In [ ]:
#np.set_printoptions(threshold=sys.maxsize) #uncomment to see max print
#plt.rcParams['figure.figsize'] = [12, 8] #change the size of figure when viewing the output
import numpy as np
import matplotlib.pyplot as plt

initial_loc = np.array([0,0])
final_loc = np.array([100,100])
sigma = np.array([[50,0],[0,50]])
mu = np.array([[10, 40], [60, 50]])

def f(x, y):
	return ((final_loc[0]-x)**2 + (final_loc[1]-y)**2)/20000 + 10000*(1/(2*np.pi*np.linalg.det(sigma)))*np.exp(-.5*(np.matmul(np.array([x-mu[0,0], y-mu[0,1]]),np.matmul(np.linalg.pinv(sigma), np.atleast_2d(np.array([x-mu[0,0], y-mu[0,1]])).T)))[0]) + 10000*(1/(2*np.pi*np.linalg.det(sigma)))*np.exp(-.5*(np.matmul(np.array([x-mu[1,0], y-mu[1,1]]),np.matmul(np.linalg.pinv(sigma), np.array([x-mu[1,0], y-mu[1,1]])))))

x = np.linspace(0, 100, 100)
y = np.linspace(0, 100, 100)
z = f(x[:,None], y[None,:])
z = np.rot90(np.fliplr(z))

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.contour3D(x, y, z, 100)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.set_title('3D Contour')
plt.show()

In [ ]:
def gradient_descent(step, precision, gx, gy, start):
    current_loc = start
    all_locs = []
    while(True):
        y = current_loc[0]
        x = current_loc[1]
        if y < 99:
            if abs(gx[x,y]) > precision:
                if gx[x,y] < 0:
                    current_loc[0] += step
                elif gx[x,y]> 0: 
                    current_loc[0] += -step
            if abs(gx[x,y]) > 2*abs(gy[x,y]):
                current_loc[0] += step
        if x < 99:
            if abs(gy[x,y]) > precision:
                if gy[x,y] < 0:
                    current_loc[1] +=step
                elif gy[x,y] > 0: 
                    current_loc[1] += -step
            if abs(gy[x,y]) > 2*abs(gx[x,y]):
                current_loc[1] += step
        all_locs.append(current_loc[:])
        if current_loc == [99,99]:
            break
    return all_locs, current_loc
            

In [ ]:
import sys
#problem 1.1(i)
plt.rcParams['figure.figsize'] = [7, 7]
X,Y = np.meshgrid(x,y)
plt.contour(X,Y,z)
plt.xlabel('x')
plt.ylabel('y')
plt.title('2D Contour and Gradient Descent')
#1.1(ii)
gy, gx = np.gradient(z)
u,v = -gx,-gy
norm = np.linalg.norm(np.array((u, v)), axis=0)   
u = u / norm    
v = v / norm
plt.quiver(X[::2,::2], Y[::2,::2], u[::2,::2], v[::2,::2], units='xy', scale=.5, color='gray')
astart = [1]
bstart = [1]
aend = [99]
bend=[99]
#1.1(iii)
plt.scatter(astart, bstart, marker=(6,2,0), color ='blue')
plt.scatter(aend, bend, marker=(6,2,0), color = 'red')
#1.2(i)
start = [0,0]
all_locs, current_loc = gradient_descent(1, .0001, gx, gy, start) #gradient function in use
all_locx = [x[0] for x in all_locs]
all_locy = [x[1] for x in all_locs]
plt.scatter(all_locx,all_locy)
plt.scatter(all_locs[0][0], all_locs[0][1], marker=(6,2,0), color = 'blue')
plt.scatter(all_locs[-1][0], all_locs[-1][1], marker=(6,2,0), color = 'red')
plt.show()
#for 1.2(ii) change starting position where gradient function is in use 
#for 1.2(iii) change mu array to get different objects


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import math
import time
import random
import heapq
import sys

# initialize parameters
height = 50; # map height
width = 50 # map width
num_bots = 15 # number of bots
max_itr = 2500 # maximum number of iterations

wall = 1 # value for a wall in the map matrix
mapped = .4 # value for a mapped area in map matrix
planned = .2 # value for a bot's planned path in map matrix
unmapped = 0 # value for a unmapped area in map matrix

# define structure to hold all information about the bots and map
bots = [{} for i in range(num_bots)]

# define the provided blueprint map
blueprint = np.pad(np.zeros((height-2,width-2)), (1,1), 'constant', constant_values = (wall, wall)) # a map provided to the bot network
blueprint[9:19,35] = wall
blueprint[9:19,44] = wall
blueprint[9,35:37] = wall
blueprint[9,42:44] = wall
blueprint[19,35:44] = wall

# define the bot's map of explored areas
explore_map = blueprint; # a map the bot network uses to keep track of where bots have visited

# initialize positions, dest, route, and exploration map
for i in range(num_bots):
    bots[i]['current_position'] = [math.floor(height/2), math.floor(width/2)] # current position of bot
    explore_map[bots[i]['current_position'][0], bots[i]['current_position'][1]] = mapped
    bots[i]['destination'] = [] # destination of bot, initialized as empty
    bots[i]['route'] = [] 

plt.imshow(explore_map, cmap='gray')
plt.title('Press \'q\' button to begin.')
plt.show()

def get_unexplored_areas(explore_map, unmapped_value):
    '''
    Write this function so that unexplored_areas is a Nx2 matrix
    where N is the number of locations the bots have not explored, and each
    row of unexplored_areas represents a location (row,col) in the map.
    We define unexplored areas as locations which have values UNMAPPED in
    explore_map. Locations with values PLANNED/MAPPED/WALL are not considered
    unexplored areas in this case. If there are no unexplored areas, then
    unexplored_areas should return empty [].
    '''
    x = np.argwhere(explore_map == unmapped_value)
    if not x.size:
        return []
    else:
        return x

def get_new_destination(current_position, unexplored_areas):
    '''
    Write this function so that it will pick the closest unexplored area
    as the new destination dest. We will keep this function simple by
    ignoring any walls that may block our path to the new destination. Here
    we define "closest" using the euclidean distance measure,
    e.g. sqrt((x1-x2)^2 + (y1-y2)^2).
    '''
    i = 0
    x = np.array([])      
    for spaces in unexplored_areas:
        norm = np.linalg.norm(current_position - unexplored_areas[i])
        i += 1
        x = np.append(x,norm)
    new_pos_index = np.where(x == min(x))
    new_pos = unexplored_areas[new_pos_index]
    return new_pos[0]


def update_explore_map(dest, route, explore_map, planned, unmapped):
    '''
    Write this function so that all the locations specified in route and dest
    are marked as PLANNED only if it was previous UNMAPPED in the explore_map
    variable.
    '''
    if explore_map[dest[0],dest[1]] == unmapped:
        explore_map[dest[0],dest[1]] = planned
    else:
        pass
    for x in route:
        if explore_map[x[0],x[1]] == unmapped:
            explore_map[x[0],x[1]] = planned
        else:
            pass
    return explore_map

def update_position(curPos, route, dest, explore_map, mapped):
    '''
    Write this function so that:
    1) update curPos 1 step closer to the destination using route
    2) mark the new location of the bot as MAPPED
    3) if the new location of the bot is at the destination, set destination
       to be empty, i.e. dest = []
    4) update the route by removing the location that the bot was just
       updated to e.g. if route was inputted as an Nx2 matrix, it should
       output as a (N-1)x2 matrix.
    '''
    curPos = route[0]
    explore_map[curPos[0],curPos[1]] = mapped
    if np.all(curPos == dest):
        dest = []
    route = np.delete(route, 0, 0)
    return curPos, route, dest, explore_map

def update_bot_info(curPos, dest, route, explore_map, botNum):
    bots[botNum]['current_position'] = curPos
    bots[botNum]['destination'] = dest
    bots[botNum]['route'] = route
    explore_map = explore_map

# heuristic function for path scoring

def heuristic(a, b):
    return np.sqrt((b[0] - a[0])**2 + (b[1] - a[1])**2)

def a_star(array, start, goal):
    
    start = tuple(start)
    goal = tuple(goal)

    neighbors = [(0,1),(0,-1),(1,0),(-1,0)]
    close_set = set()
    came_from = {}
    gscore = {start:0}
    fscore = {start:heuristic(start, goal)}

    oheap = []
    heapq.heappush(oheap, (fscore[start], start))
 
    while oheap:

        current = heapq.heappop(oheap)[1]

        if current == goal:
            data = []
            while current in came_from:
                data.append(list(current))
                current = came_from[current]
            return np.array(data)

        close_set.add(current)
        for i, j in neighbors:
            neighbor = current[0] + i, current[1] + j

            tentative_g_score = gscore[current] + heuristic(current, neighbor)

            if 0 <= neighbor[0] < array.shape[0]:
                if 0 <= neighbor[1] < array.shape[1]:                
                    if array[neighbor[0]][neighbor[1]] == 1:
                        continue
                else:
                    # array bound y walls
                    continue
            else:
                # array bound x walls
                continue

            if neighbor in close_set and tentative_g_score >= gscore.get(neighbor, 0):
                continue
 
            if  tentative_g_score < gscore.get(neighbor, 0) or neighbor not in [i[1]for i in oheap]:

                came_from[neighbor] = current
                gscore[neighbor] = tentative_g_score
                fscore[neighbor] = tentative_g_score + heuristic(neighbor, goal)
                heapq.heappush(oheap, (fscore[neighbor], neighbor))
 
    return False

for itr in range(max_itr):
    for botNum in range(num_bots):
        # extract bot's info for convenience/code readability
        curPos = bots[botNum]['current_position']
        dest = bots[botNum]['destination']
        route = bots[botNum]['route']

        # if the bot doesn't have a destination then pick a new destination
        if len(dest) == 0: 
                    
            # get the locations of all areas that are labeled as unmapped 
            # TODO: write this function above.
            unexplored_areas = get_unexplored_areas(explore_map, unmapped)
            
            # if there are no more unexplored areas, then this bot stops moving
            if len(unexplored_areas) == 0:
                dest = []
                route = []
                update_bot_info(curPos, dest, route, explore_map, botNum)
                continue
            
            # calculate bot's new destination (TODO: write this function above)
            dest = get_new_destination(curPos, unexplored_areas)
            
            # calculates bot's route to the destination (you do not need to worry about this function)
            if a_star(explore_map, curPos, dest).size:
                route = a_star(explore_map, curPos, dest)
                route = np.vstack((route, curPos))
                route = route[::-1]

            
            # mark the location in explored_map as planned if it was unmapped (TODO: write this function above)
            explore_map = update_explore_map(dest, route, explore_map, planned, unmapped)

        # Using the calculated route, move the bot 1 step towards the destination bot's destination (TODO: write this function above)
        curPos, route, dest, explore_map = update_position(curPos, route, dest, explore_map, mapped)
        
        # update bot's curr position, past position, destination, and map
        update_bot_info(curPos, dest, route, explore_map, botNum)

    # update display
    plt.imshow(explore_map, cmap='gray')
    for i in range(num_bots):
        fig = plt.gcf()
        ax = fig.gca()
        ax.add_patch(patches.Rectangle((bots[i]['current_position'][1]-.5, bots[i]['current_position'][0]-.5), 1, 1, edgecolor = 'blue', facecolor = 'red', fill=True))
        if len(bots[i]['destination']) != 0:
            plt.plot([b[1] for b in bots[i]['route']], [b[0] for b in bots[i]['route']], color='red')
            plt.plot(bots[i]['destination'][1], bots[i]['destination'][0], color='yellow')

    plt.show(block=False)
    plt.pause(.25)
    ax.patches = []
    mapped_count = sum(sum(np.array(explore_map) == mapped))
    wall_count = sum(sum(np.array(explore_map) == wall))
    explore_size = explore_map.size
    if (mapped_count + wall_count) == explore_size:
        break

print("Number of iterations: "+str(itr))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def forwardKinematics(theta0, theta1, theta2, l0, l1, l2):
    y_1 = np.sin(theta0)*l0
    x_1 = np.cos(theta0)*l0
    x_2 = l0*np.cos(theta0)+l1*np.cos(theta0+theta1)
    y_2 = l0*np.sin(theta0)+l1*np.sin(theta0+theta1)
    x_e = l0*np.cos(theta0)+l1*np.cos(theta0+theta1)+l2*np.cos(theta0+theta1-theta2)
    y_e = l0*np.sin(theta0)+l1*np.sin(theta0+theta1)+l2*np.sin(theta0+theta1-theta2)
    drawRobot(x_1,y_1,x_2,y_2,x_e,y_e)
    return x_1, y_1, y_2, x_2, y_e, x_e

def inverseKinematics(l0,l1,l2,x_e_target,y_e_target):

    # Initialize for the plots:
    end_effector_positions = []

    # Initialize the thetas to some value
    theta0 = np.pi/3
    theta1 = 0
    theta2 = 0
    
    # Obtain end effector position x_e, y_e for current thetas: 
    # HINT: use your ForwardKinematics function   

    x_1, y_1, y_2, x_2, y_e, x_e = forwardKinematics(theta0, theta1, theta2, l0, l1, l2)
    condition = False 
    while condition==False: # Replace the '1' with a condition that checks if your estimated [x_e,y_e] is close to [x_e_target,y_e_target]
        if abs(x_e_target-x_e)< .001 and abs(y_e_target-y_e)< .001:
            condition = True
            
        # Calculate the Jacobian matrix for current values of theta
        # HINT: write a function for doing this      
        def jacobian(theta0,theta1,theta2,l0,l1,l2):
            jacob = np.array([])
            dx1 = -l0*np.sin(theta0)-l1*np.sin(theta0+theta1)-l2*np.sin(theta0+theta1-theta2)
            dx2 = -l1*np.sin(theta0+theta1)-l2*np.sin(theta0+theta1-theta2)
            dx3 = l2*np.sin(theta0+theta1-theta2)
            dy1 = l0*np.cos(theta0)+l1*np.cos(theta0+theta1)+l2*np.cos(theta0+theta1-theta2)
            dy2 = l1*np.cos(theta0+theta1)+l2*np.cos(theta0+theta1-theta2)
            dy3 = -l2*np.cos(theta0+theta1-theta2)
            j1 = np.array([dx1, dx2, dx3])
            j2 = np.array([dy1, dy2, dy3])
            jacob = np.array([j1,j2])
            return jacob
        jacob = jacobian(theta0,theta1,theta2,l0,l1,l2)
        # Calculate the pseudo-inverse of the jacobian (HINT: numpy pinv())     
        np.linalg.pinv(jacob)

        # Update the values of the thetas by a small step
        jinv = np.linalg.pinv(jacob)
        x_end = x_e_target - x_e
        y_end = y_e_target - y_e
        delta_end = np.array([x_end,y_end])
        delta_thetas = np.matmul(jinv,delta_end)*.05
        current_thetas = np.array([theta0,theta1,theta2])
        theta_targets = delta_thetas + current_thetas
        theta0 = theta_targets[0]
        theta1 = theta_targets[1]
        theta2 = theta_targets[2]

        # Obtain end effector position x_e, y_e for the updated thetas:
        x_1, y_1, y_2, x_2, y_e, x_e = forwardKinematics(theta0, theta1, theta2, l0, l1, l2)
        # If you would like to visualize the iterations, draw the robot using drawRobot. 
        drawRobot(x_1,y_1,x_2,y_2,x_e,y_e)
        
        # Save end effector positions for the plot:
        position =[x_e,y_e]
        end_effector_positions.append(position)

    # Plot the final robot pose
    drawRobot(x_1,y_1,x_2,y_2,x_e,y_e)

    # Plot the end effector position through the iterations
    i = 0
    x = []
    y = []
    for items in end_effector_positions:
        x.append(items[0])
        y.append(items[1])
        i += 0
    plt.scatter(x,y)
    plt.show
    return x_e,y_e,theta0,theta1,theta2
    

def drawRobot(x_1,y_1,x_2,y_2,x_e,y_e):
    x_0, y_0 = 0, 0
    plt.plot([x_0, x_1, x_2, x_e], [y_0, y_1, y_2, y_e], lw=4.5)
    plt.scatter([x_0, x_1, x_2, x_e], [y_0, y_1, y_2, y_e], color='r')
    plt.axis('square')
    plt.show()

In [ ]:
x_1, y_1, y_2, x_2, y_e, x_e = forwardKinematics(np.pi/3, np.pi/12, -np.pi/6, 3, 5, 7)


In [ ]:
inverseKinematics(10,10,10,6,12)

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np

def draw_path(final_path_points, other_path_points):
    '''
    final_path_points: the list of points (as tuples or lists) comprising your final maze path. 
    other_path_points: the list of points (as tuples or lists) comprising all other explored maze points. 
    (0,0) is the start, and (49,49) is the goal.
    Note: the maze template must be in the same folder as this script.
    '''
    im = plt.imread('172maze2021.png')
    x_interval = (686-133)/49
    y_interval = (671-122)/49
    plt.imshow(im)
    fig = plt.gcf()
    ax = fig.gca()
    circle_start = plt.Circle((133,800-122), radius=4, color='lime')
    circle_end = plt.Circle((686, 800-671), radius=4, color='red')
    ax.add_patch(circle_start)
    ax.add_patch(circle_end)
    for point in other_path_points:
        if not (point[0]==0 and point[1]==0) and not (point[0]==49 and point[1]==49):
            circle_temp = plt.Circle((133+point[0]*x_interval, 800-(122+point[1]*y_interval)), radius=4, color='blue')
            ax.add_patch(circle_temp)
    for point in final_path_points:
        if not (point[0]==0 and point[1]==0) and not (point[0]==49 and point[1]==49):
            circle_temp = plt.Circle((133+point[0]*x_interval, 800-(122+point[1]*y_interval)), radius=4, color='yellow')
            ax.add_patch(circle_temp)
    plt.show()
    ### Your Work Below: 

In [ ]:
picklemaze = []
import pickle
with(open('172maze2021.p','rb')) as file:
    while True:
        try:
            picklemaze.append(pickle.load(file))
        except EOFError:
            break
picklemaze=picklemaze[0]

def sense_maze(current_node,picklemaze):
    sense = picklemaze.get(current_node)
    neighbors = []
    current = list(current_node)
    if sense[0] == True:
        add = (current[0],current[1]+1)
        neighbors.append(add)
    if sense[1] == True:
        add = (current[0]+1,current[1])
        neighbors.append(add)
    if sense[2] == True:
        add = (current[0],current[1]-1)
        neighbors.append(add)
    if sense[3] == True:
        add = (current[0]-1,current[1])
        neighbors.append(add)
    return neighbors    
    
def DFS(picklemaze):
    stack = []
    visited_nodes = []
    parent = {}
    starting = (0,0)
    stack.append(starting)
    current_node = stack[-1]
    iteration = 0
    while current_node != (49,49):
        if current_node not in visited_nodes:
            visited_nodes.append(current_node)
        neighbors = sense_maze(current_node,picklemaze)
        for i in neighbors:
            if i not in visited_nodes:
                stack.append(i)
                parent[i] = current_node
                break
        if stack[-1] == current_node:
            stack.pop()       
            
        draw_path([current_node], visited_nodes)
        current_node = stack[-1]
        iteration += 1 
    main_path = [(49,49)]
    while main_path[-1] != starting:
        main_path.append(parent[(main_path[-1])])
    draw_path(main_path,visited_nodes)
    
    return iteration, parent
                    
        
def BFS(picklemaze):
    queue = []
    visited_nodes = []
    parent = {}
    starting = (0,0)
    queue.append(starting)
    current_node = queue[-1]
    end = (49,49)
    iteration = 0
    while current_node != (49,49):
        neighbors = sense_maze(current_node,picklemaze)
        for i in neighbors:
            if i not in visited_nodes:
                visited_nodes.append(i)
                queue.append(i)
                parent[i]=current_node          
        queue.pop(0)
        current_node = queue[0]     
        draw_path(queue, visited_nodes)
        iteration += 1
        
    main_path = [end]
    while main_path[-1] != starting:
        main_path.append(parent[(main_path[-1])])
    draw_path(main_path,visited_nodes)
    
    return iteration
            
   
    

In [ ]:
np.set_printoptions(threshold=sys.maxsize)
DFS(picklemaze)

In [ ]:
BFS(picklemaze)